<a href="https://colab.research.google.com/github/soccer1356-2000/Ingong_MidtermExam/blob/main/%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C_201920778_%EC%9D%B4%EC%8A%B9%EC%9C%A4_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
from torchvision import datasets, transforms # Fashion-MNIST dataset for PyTorch
# Download and load the FashionMNIST training data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = datasets.FashionMNIST('MNIST_data/', download = True, train = True,
transform = transform)
testset = datasets.FashionMNIST('MNIST_data/', download = True, train = False,
transform = transform)

In [21]:
import torch
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [6]:
trainset # 28 * 28 * 1

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: MNIST_data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [27]:
import torch.nn as nn
import torch.nn.functional as F

In [13]:
def conv3x3(in_planes, out_planes, stride=1, groups=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, groups=groups, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [40]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64,  norm_layer=nn.BatchNorm2d):
      
        super().__init__()
            
        self.conv1 = conv3x3(inplanes, planes, stride)  
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)

        return out

In [38]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, norm_layer=nn.BatchNorm2d):
        super().__init__()

        # ResNext
        width = int(planes * (base_width / 64.)) * groups
        

        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups) # conv2에서 downsample
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        # 1x1 convolution layer
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        # 3x3 convolution layer
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        # 1x1 convolution layer
        out = self.conv3(out)
        out = self.bn3(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [57]:
class ResNet(nn.Module):
  def __init__(self, block, layers, zero_init_residual=False,
                 groups=1, width_per_group=64, norm_layer=nn.BatchNorm2d):
    super().__init__()
    self.norm_layer = norm_layer
    self.groups = groups
    self.base_width = width_per_group

    self.inplane = 64

    #stage 1
    self.cov1 = nn.Conv2d(1,self.inplane,kernel_size=7,stride = 2, padding = 3, bias = False)
    self.bn1 = norm_layer(self.inplane)
    self.relu = nn.ReLU(inplace = True)
    self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    #stage 2
    self.layer1 = self.make_layer(block,64,layers[0])
    #stage 3
    self.layer2 = self.make_layer(block,128,layers[1],stride=2)
    #stage 4
    self.layer3 = self.make_layer(block,256,layers[2],stride=2)
    #stage 5
    self.layer4 = self.make_layer(block,512,layers[3],stride=2)
    self.averpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(521 * block.expansion,10)
  
  def make_layer(self,block,palnes,blocks,stride = 1):
    norm_layer = self.norm_layer

    down_channel = None
    
    if stride != 1:
      down_channel = nn.Sequential(conv1x1(self.inplane, palnes * block.expansion,stride),norm_layer(palnes * block.expansion))

    layers = []

    layers.append(block(self.inplane, palnes, stride, down_channel, self.groups,self.base_width, norm_layer))
    self.inplane = palnes * block.expansion # inplanes 업데이트
    for _ in range(1, blocks):
        layers.append(block(self.inplane, palnes, groups=self.groups,base_width=self.base_width, norm_layer=norm_layer))

    return nn.Sequential(*layers)

  def forward(self, x):

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    x = self.fc(x)

    output = nn.log_softmax(x, dim=1)
    return x

In [23]:
from torch.autograd import Variable
import torch.optim as optim

In [18]:
args={}
kwargs={}
args['batch_size']=1000
args['test_batch_size']=1000
args['epochs']=10  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=5000 // args['batch_size']
args['cuda']=False

In [54]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.data))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in testloader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(testloader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

In [58]:
model = ResNet(BasicBlock,[2,2,2,2])
if args['cuda']:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()

RuntimeError: ignored